# Ramsey King
# DSC 550 - Data Mining
# September 9, 2021
# Week 2: Handling Categorical Data, Text, Dates & Times
# Exercise 2.2 


You can find the dataset controversial-comments.jsonl for this exercise in the Weekly Resources: Week 2 Data Files.
Pre-processing Text: For this part, you will start by reading the controversial-comments.jsonl file into a DataFrame.

In [17]:
import pandas as pd
import numpy as np

comments_df = pd.read_json('controversial-comments.jsonl', lines=True)
comments_df.head()
# I am going to limit the number of entries from 950,000 to 50,000 as per the instructions in the assignment to help with the runtime.

comments_df = comments_df.sample(50000).copy()


A. Convert all text to lowercase letters.

In [18]:
comments_df['txt'] = comments_df['txt'].apply(str.lower)
comments_df['txt'][:5]

354385    not at all, i'm all for automation and the eas...
475928                                        seems correct
935922    no it's not.  \n\nvote for your guy in the pri...
188698    &gt; famous person voices their opinion\n\nnot...
560119    **the legislative branch** of government wrote...
Name: txt, dtype: object

B. Remove all punctuation from the text.


In [19]:
import re
comments_df['txt'] = comments_df['txt'].apply(lambda remove_punct: re.sub(r'[^\w\s]', '', remove_punct))
comments_df['txt'][:5]

354385    not at all im all for automation and the ease ...
475928                                        seems correct
935922    no its not  \n\nvote for your guy in the prima...
188698    gt famous person voices their opinion\n\nnothi...
560119    the legislative branch of government wrote the...
Name: txt, dtype: object

C. Remove stop words.


In [20]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = stopwords.words('english')

def rem_stop_words(rem_list):
    return [word for word in rem_list if word not in stop_words]

comments_df['txt_tokenized'] = comments_df['txt'].apply(word_tokenize)
comments_df['txt_tokenized'] = comments_df['txt_tokenized'].apply(rem_stop_words)
comments_df['txt_tokenized'][:5]

354385    [im, automation, ease, bring, lives, problem, ...
475928                                     [seems, correct]
935922    [vote, guy, primary, push, representatives, pa...
188698    [gt, famous, person, voices, opinion, nothing,...
560119    [legislative, branch, government, wrote, law, ...
Name: txt_tokenized, dtype: object

D. Apply NLTK’s PorterStemmer.

In [21]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def apply_porter_stemmer(tokenized_words):
    return [porter.stem(word) for word in tokenized_words]

comments_df['txt_tokenized_stemmed'] = comments_df['txt_tokenized'].apply(apply_porter_stemmer)
comments_df['txt_tokenized_stemmed'][:5]

354385    [im, autom, eas, bring, live, problem, current...
475928                                      [seem, correct]
935922    [vote, guy, primari, push, repres, parti, alig...
188698    [gt, famou, person, voic, opinion, noth, best,...
560119    [legisl, branch, govern, wrote, law, accident,...
Name: txt_tokenized_stemmed, dtype: object

Now that the data is pre-processed, you will apply three different techniques to get it into a usable form for
model-building. Apply each of the following steps (individually) to the pre-processed data.

A. Convert each text entry into a word-count vector (see sects 5.3 & 6.8 in the Machine Learning with Python Cookbook).

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
results = count.fit_transform(comments_df['txt'])
print(comments_df.info())
print(type(results))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 354385 to 853888
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   con                    50000 non-null  int64 
 1   txt                    50000 non-null  object
 2   txt_tokenized          50000 non-null  object
 3   txt_tokenized_stemmed  50000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.9+ MB
None
<class 'scipy.sparse.csr.csr_matrix'>


B. Convert each text entry into a part-of-speech tag vector (see section 6.7 in the Machine Learning with Python Cookbook).

In [23]:
from nltk import pos_tag
from nltk import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
# nltk.download('averaged_perceptron_tagger')
tweets = comments_df['txt'].tolist()
tagged_tweets = []
for tweet in tweets:
    tweet_tag = nltk.pos_tag(word_tokenize(tweet))
    tagged_tweets.append([tag for word, tag in tweet_tag])

one_hot_multi = MultiLabelBinarizer()
one_hot_multi.fit_transform(tagged_tweets)


array([[0, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

C. Convert each entry into a term frequency-inverse document frequency (tfidf) vector
(see section 6.9 in the Machine Learning with Python Cookbook).

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_data = np.array(tweets)
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text_data)
tfidf.vocabulary_

{'not': 28102,
 'at': 4049,
 'all': 2702,
 'im': 21393,
 'for': 15441,
 'automation': 4260,
 'and': 3049,
 'the': 40224,
 'ease': 12745,
 'it': 22774,
 'will': 44518,
 'bring': 6197,
 'to': 40844,
 'our': 29092,
 'lives': 24577,
 'only': 28790,
 'problem': 31667,
 'is': 22689,
 'that': 40191,
 'we': 44017,
 'currently': 10098,
 'have': 18469,
 'no': 27770,
 'system': 39619,
 'in': 21606,
 'place': 30552,
 'help': 18673,
 'millions': 26214,
 'of': 28535,
 'people': 30026,
 'would': 44850,
 'be': 4830,
 'out': 29100,
 'job': 23076,
 'means': 25726,
 'support': 39292,
 'themselves': 40274,
 'universal': 42559,
 'basic': 4739,
 'income': 21715,
 'fix': 15122,
 'issue': 22765,
 'but': 6577,
 'current': 10097,
 'capitalistic': 6912,
 'society': 37268,
 'never': 27587,
 'accept': 1947,
 'there': 40318,
 'are': 3651,
 'already': 2810,
 'freak': 15727,
 'about': 1845,
 'welfare': 44143,
 'queens': 32462,
 'entitled': 13528,
 'millennials': 26197,
 'who': 44393,
 'just': 23260,
 'want': 43831,
 

For the three techniques in problem (2) above, give an example where each would be useful.

Word count vector - can help identify a subject of a text and a hidden subjet within text.  Can also be used for sentiment analysis - THere are references out there that use sentiment analysis to their benefit.  One example would be an airline company that reads comments from Twitter.  If the airline company identifies certain words that have a negative sentiment, they can look out for these words and make marketing adjustments as necessary to help with customer satisfaction.

Part of speech tag vector - Tagging parts of speech is important in that it allows for the context of a word to be identified.  For example, if we continue with the twitter comment airline example, the word "late" without any context would probably be viewed as negative when it applies to an airline company.  For example, if a comment was " #airlinecompany flight 1234 is running late as usual!" then the word "late" in this context has a negative sentiment.  However, if another comment was "#airlinecompany made sure that I made my flight even though I got to the airport really late.", then that can be viewed positively.

Term frequency - inverse document frequency - One of the most important uses of term frequency - inverse document frequency is found in something we use almost every day, Google search.  When we search for a term in Google, the words we search are compared with all the text that Google is able to search in its universe.  Those terms are compared with the texts Google searches upon, and by using the Term Frequency - Inverse Document Frequency, it is able to return the most relevant documents (websites) to our terms.